In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas import DataFrame
import argparse
import time
from selenium import webdriver
import numpy as np
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
YEAR = 2014

In [3]:
link = f'https://www.atptour.com/en/stats/leaderboard?boardType=pressure&timeFrame={YEAR}&surface=hard&versusRank=all&formerNo1=false'

options = webdriver.ChromeOptions()
options.add_argument('--headless=new')
options.add_argument(f'user-agent={YEAR}-{str(YEAR)[::-1]}')

browser = webdriver.Chrome(service=Service(
    ChromeDriverManager().install()), options=options)
browser.get(link)

html = browser.page_source

In [4]:
soup = BeautifulSoup(html, 'html.parser')
player_tbls = soup.find(id="statsListingTableContent").find('table')


browser.quit()

In [5]:
thead_child = player_tbls.find('thead').find_all(recursive=False)

tbody_child = player_tbls.find('tbody').find_all(recursive=False)


thead = player_tbls.find('thead')
for child in thead_child:
    thead.append(child)

tbody = player_tbls.find('tbody')
for child in tbody_child:
    tbody.append(child)

In [6]:
df = pd.read_html(str(player_tbls))[0]
# print(df)
df.to_csv(f'all_pres_{YEAR}.csv', index=False)

df.columns = ['rank', 'name', 'pressure_rating', 'break_pts_converted%',
              'break_pts_saved%', 'tie_breaks%', 'deciding_sets%']

main_df = pd.read_csv(f'./data/aus-open-player-stats-{YEAR}.csv')
main_df = main_df.merge(df, left_on='name', right_on='name', how='left')

In [7]:
main_df

,name,year,aces,double_faults,1st_serve,1st_serve_points_won,2nd_serve_points_won,break_points_faced,break_points_saved,service_games_played,...,total_points_won,rank_x,height (cm),matches_played,rank_y,pressure_rating,break_pts_converted%,break_pts_saved%,tie_breaks%,deciding_sets%
0,Novak Djokovic,2014,269,75,67%,76%,56%,174,64%,540,...,56%,2,188,42,5.0,246.1,47.2%,64.4%,54.5%,80.0%
1,Aljaz Bedene,2014,69,28,53%,74%,45%,56,55%,83,...,49%,105,183,17,NaN,NaN,NaN,NaN,NaN,NaN
2,Roger Federer,2014,438,90,62%,79%,58%,238,71%,717,...,55%,6,185,52,1.0,258.8,39.5%,70.6%,65.4%,83.3%
3,Yen-Hsun Lu,2014,140,97,63%,72%,52%,205,65%,380,...,51%,62,180,3,34.0,202.4,41.5%,65.4%,45.5%,50.0%
4,Rafael Nadal,2014,87,40,70%,73%,57%,117,64%,307,...,54%,1,185,17,10.0,238.0,44.2%,64.1%,58.3%,71.4%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,Tobias Kamke,2014,31,43,58%,65%,52%,117,60%,169,...,48%,77,178,42,NaN,NaN,NaN,NaN,NaN,NaN
124,Viktor Troicki,2014,86,21,54%,81%,50%,37,59%,92,...,53%,79,193,19,NaN,NaN,NaN,NaN,NaN,NaN
125,Jiri Vesely,2014,85,79,60%,71%,49%,109,61%,163,...,49%,82,198,17,NaN,NaN,NaN,NaN,NaN,NaN
126,Donald Young,2014,87,107,62%,68%,49%,214,57%,362,...,49%,96,183,25,33.0,202.8,36.5%,56.5%,63.6%,46.2%


In [8]:
link = f'https://www.atptour.com/en/stats/leaderboard?boardType=pressure&timeFrame=Career&surface=hard&versusRank=all&formerNo1=false'

options = webdriver.ChromeOptions()
options.add_argument('--headless=new')

browser = webdriver.Chrome(service=Service(
    ChromeDriverManager().install()), options=options)
browser.get(link)

In [9]:
html = browser.page_source


# page = requests.get(link, headers={'user-agent': f'my-app/{YEAR}'})
soup = BeautifulSoup(html, 'html.parser')
player_tbls = soup.find(id="statsListingTableContent").find('table')


browser.quit()

In [10]:
thead_child = player_tbls.find('thead').find_all(recursive=False)

tbody_child = player_tbls.find('tbody').find_all(recursive=False)

thead = player_tbls.find('thead')
for child in thead_child:
    thead.append(child)

tbody = player_tbls.find('tbody')
for child in tbody_child:
    tbody.append(child)

In [11]:
df = pd.read_html(str(player_tbls))[0]


no_stats = main_df.loc[main_df['pressure_rating'].isna(), 'name']

df.columns = ['rank', 'name', 'pressure_rating', 'break_pts_converted%',
              'break_pts_saved%', 'tie_breaks%', 'deciding_sets%']

df = df[df['name'].isin(no_stats)].copy(deep=True).reset_index(drop=True)


# main_df = main_df.merge(df, left_on='name', right_on='name', how='left')
df

,rank,name,pressure_rating,break_pts_converted%,break_pts_saved%,tie_breaks%,deciding_sets%
0,48,Lleyton Hewitt,220.2,43.2%,61.7%,50.0%,65.3%
1,91,Lucas Pouille,214.0,39.6%,63.8%,49.4%,61.2%
2,115,Gilles Muller,210.7,34.6%,66.9%,54.5%,54.7%
3,126,Marcos Baghdatis,208.8,40.7%,59.6%,57.1%,51.4%
4,148,Viktor Troicki,206.9,41.3%,60.0%,50.0%,55.6%
...,...,...,...,...,...,...,...
56,531,Elias Ymer,162.1,37.0%,54.9%,23.1%,47.1%
57,542,Igor Sijsling,159.2,36.6%,60.0%,36.8%,25.8%
58,561,Jimmy Wang,150.7,41.9%,59.3%,41.2%,8.3%
59,565,Marsel Ilhan,147.5,37.6%,57.7%,22.2%,30.0%


In [12]:
# main_df[main_df.columns[24:]] = main_df[main_df.columns[24:]].replace(np.nan, 0)

In [13]:
main_df

,name,year,aces,double_faults,1st_serve,1st_serve_points_won,2nd_serve_points_won,break_points_faced,break_points_saved,service_games_played,...,total_points_won,rank_x,height (cm),matches_played,rank_y,pressure_rating,break_pts_converted%,break_pts_saved%,tie_breaks%,deciding_sets%
0,Novak Djokovic,2014,269,75,67%,76%,56%,174,64%,540,...,56%,2,188,42,5.0,246.1,47.2%,64.4%,54.5%,80.0%
1,Aljaz Bedene,2014,69,28,53%,74%,45%,56,55%,83,...,49%,105,183,17,NaN,NaN,NaN,NaN,NaN,NaN
2,Roger Federer,2014,438,90,62%,79%,58%,238,71%,717,...,55%,6,185,52,1.0,258.8,39.5%,70.6%,65.4%,83.3%
3,Yen-Hsun Lu,2014,140,97,63%,72%,52%,205,65%,380,...,51%,62,180,3,34.0,202.4,41.5%,65.4%,45.5%,50.0%
4,Rafael Nadal,2014,87,40,70%,73%,57%,117,64%,307,...,54%,1,185,17,10.0,238.0,44.2%,64.1%,58.3%,71.4%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,Tobias Kamke,2014,31,43,58%,65%,52%,117,60%,169,...,48%,77,178,42,NaN,NaN,NaN,NaN,NaN,NaN
124,Viktor Troicki,2014,86,21,54%,81%,50%,37,59%,92,...,53%,79,193,19,NaN,NaN,NaN,NaN,NaN,NaN
125,Jiri Vesely,2014,85,79,60%,71%,49%,109,61%,163,...,49%,82,198,17,NaN,NaN,NaN,NaN,NaN,NaN
126,Donald Young,2014,87,107,62%,68%,49%,214,57%,362,...,49%,96,183,25,33.0,202.8,36.5%,56.5%,63.6%,46.2%


In [14]:
main_df.set_index('name', inplace=True)
df.set_index('name', inplace=True)

main_df['pressure_rating'].fillna(df['pressure_rating'], inplace=True)
main_df.reset_index(inplace=True)

In [15]:
main_df

,name,year,aces,double_faults,1st_serve,1st_serve_points_won,2nd_serve_points_won,break_points_faced,break_points_saved,service_games_played,...,total_points_won,rank_x,height (cm),matches_played,rank_y,pressure_rating,break_pts_converted%,break_pts_saved%,tie_breaks%,deciding_sets%
0,Novak Djokovic,2014,269,75,67%,76%,56%,174,64%,540,...,56%,2,188,42,5.0,246.1,47.2%,64.4%,54.5%,80.0%
1,Aljaz Bedene,2014,69,28,53%,74%,45%,56,55%,83,...,49%,105,183,17,NaN,199.6,NaN,NaN,NaN,NaN
2,Roger Federer,2014,438,90,62%,79%,58%,238,71%,717,...,55%,6,185,52,1.0,258.8,39.5%,70.6%,65.4%,83.3%
3,Yen-Hsun Lu,2014,140,97,63%,72%,52%,205,65%,380,...,51%,62,180,3,34.0,202.4,41.5%,65.4%,45.5%,50.0%
4,Rafael Nadal,2014,87,40,70%,73%,57%,117,64%,307,...,54%,1,185,17,10.0,238.0,44.2%,64.1%,58.3%,71.4%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,Tobias Kamke,2014,31,43,58%,65%,52%,117,60%,169,...,48%,77,178,42,NaN,165.9,NaN,NaN,NaN,NaN
124,Viktor Troicki,2014,86,21,54%,81%,50%,37,59%,92,...,53%,79,193,19,NaN,206.9,NaN,NaN,NaN,NaN
125,Jiri Vesely,2014,85,79,60%,71%,49%,109,61%,163,...,49%,82,198,17,NaN,186.4,NaN,NaN,NaN,NaN
126,Donald Young,2014,87,107,62%,68%,49%,214,57%,362,...,49%,96,183,25,33.0,202.8,36.5%,56.5%,63.6%,46.2%


In [16]:
main_df.to_csv(f'data/aus-open-player-stats-{YEAR}.csv', index=False)

In [17]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 29 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   name                         128 non-null    object 
 1   year                         128 non-null    int64  
 2   aces                         128 non-null    int64  
 3   double_faults                128 non-null    int64  
 4   1st_serve                    128 non-null    object 
 5   1st_serve_points_won         128 non-null    object 
 6   2nd_serve_points_won         128 non-null    object 
 7   break_points_faced           128 non-null    int64  
 8   break_points_saved           128 non-null    object 
 9   service_games_played         128 non-null    int64  
 10  service_games_won            128 non-null    object 
 11  total_service_points_won     128 non-null    object 
 12  1st_serve_return_points_won  128 non-null    object 
 13  2nd_serve_return_poi